# **PySpark**: The Apache Spark Python API

## 1. Introduction

This notebook shows how to connect Jupyter notebooks to a Spark cluster to process data using Spark Python API.

## 2. The Spark Cluster

### 2.1. Connection

To connect to the Spark cluster, create a SparkSession object with the following params:

+ **appName:** application name displayed at the [Spark Master Web UI](http://localhost:8080/);
+ **master:** Spark Master URL, same used by Spark Workers;
+ **spark.executor.memory:** must be less than or equals to docker compose SPARK_WORKER_MEMORY config.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

More confs for SparkSession object in standalone mode can be added using the **config** method. Checkout the API docs [here](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession).

## 3. The Data

### 3.1. Introduction

We will be using Spark Python API to read, process and write data. Checkout the API docs [here](https://spark.apache.org/docs/latest/api/python/index.html).

### 3.2. Read

Let's read the data concerning Biomass energy production ([source](https://www.kaggle.com/datasets/parisrohan/credit-score-classification)) from the cluster's simulated **Spark standalone cluster** into a Spark dataframe.
This dataset shows multiple information related to the details of more than 2 millions biomass.

In [5]:
data = spark.read.csv(path="data/generated_2mill_data.csv", sep=",", header=True)

Let's then display some dataframe metadata, such as the number of rows and cols and its schema (cols name and type).

In [6]:
data.count()

2000000

In [5]:
len(data.columns)

8

Let s see the types of each column in our data

In [6]:
data.dtypes

[('ID', 'string'),
 ('name', 'string'),
 ('Moisture content', 'string'),
 ('Volatile matter', 'string'),
 ('Fixed carbon', 'string'),
 ('Carbon', 'string'),
 ('Hydrogen', 'string'),
 ('Net calorific value (LHV)', 'string')]

We can clearly see that all columns are "string" type. It is necessary to change some of the columns into "float" before proceding to modeling

### 3.3. Process

The columns below are supposed to be numbers.

We ll create a function that converts the selected columns into "float"

In [10]:
columns = data.columns[2:8]
print(columns)


['Moisture content', 'Volatile matter', 'Fixed carbon', 'Carbon', 'Hydrogen', 'Net calorific value (LHV)']


In [11]:
from pyspark.sql.functions import col

def convert_to_float(df, column):
    return df.withColumn(column, col(column).cast("float"))

# Apply the conversion to each column
for column in columns:
    data = convert_to_float(data, column)

# Show the schema to verify the changes
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Moisture content: float (nullable = true)
 |-- Volatile matter: float (nullable = true)
 |-- Fixed carbon: float (nullable = true)
 |-- Carbon: float (nullable = true)
 |-- Hydrogen: float (nullable = true)
 |-- Net calorific value (LHV): float (nullable = true)



Let s check the usual statistiques of our columns of the type "float"

In [17]:
summary_stats = data[columns].describe()
summary_stats.show()

+-------+------------------+-----------------+------------------+------------------+------------------+-------------------------+
|summary|  Moisture content|  Volatile matter|      Fixed carbon|            Carbon|          Hydrogen|Net calorific value (LHV)|
+-------+------------------+-----------------+------------------+------------------+------------------+-------------------------+
|  count|           2050118|          2050118|           2050118|           2050118|           2050118|                  2050118|
|   mean| 21.59959719049037|56.61321345239012| 19.04222656823365|40.768696074712544|4.6419355674140945|       14.847574745704133|
| stddev|15.321312125080546|20.37650947685937|12.204060212590859| 15.11232645800606|1.7320383906153491|        6.994227433671838|
|    min|     1.09942885E-5|     3.6880007E-5|      6.7279407E-6|      5.0687526E-5|      2.7098176E-5|                -9.005806|
|    max|         115.54109|        157.13931|          90.92725|         113.17256|      

<h1>Interpretation</h1>

By observing the table above, we can find many unusual values.

For example:
the mean of the column "Age" is 124.25.

the mean of the columns "Num_Bank_Accounts" is 16.57.

there are many values that seems much bigger than they should be, and that is due to mistyping and incorrect data that could ve been wrongly registered for various reasons.

<h5>Our job is to fix this data before creating a model</h5>
Let s create a boxplot for each columns to check the distrubution of values in each column


In [22]:
negative_values_count = data.filter(col('Net calorific value (LHV)') < 0).count()
print(negative_values_count)

34254


Let s remove the rows where the "net calorific value" is negative

In [35]:
data = data.filter(col('Net calorific value (LHV)') >= 0)
net_cal_val_stats = data.describe("Net calorific value (LHV)")
net_cal_val_stats.show()
print(data.count())

+-------+-------------------------+
|summary|Net calorific value (LHV)|
+-------+-------------------------+
|  count|                  2015864|
|   mean|       15.131477251125487|
| stddev|        6.700126756281459|
|    min|             1.7995875E-4|
|    max|                 50.16783|
+-------+-------------------------+



2015864


Now, our data is cleaned (at the cost of 30 000 rows)

<h3>Correlation matrix</h3>

In [37]:
from pyspark.sql.functions import corr
from itertools import combinations

# Create an empty dictionary to store the correlation values
correlation_dict = {}

# Use combinations to get unique pairs of columns
for col1, col2 in combinations(columns, 2):
    correlation = data.stat.corr(col1, col2)
    correlation_dict[(col1, col2)] = correlation

# Display the correlation matrix
for key, value in correlation_dict.items():
    print(f"Correlation between {key[0]} and {key[1]}: {value}")


Correlation between Moisture content and Volatile matter: -0.0017847007091230658
Correlation between Moisture content and Fixed carbon: -0.0009742749881004561
Correlation between Moisture content and Carbon: 0.005826871260981072
Correlation between Moisture content and Hydrogen: 0.001056513892655901
Correlation between Moisture content and Net calorific value (LHV): -0.05361232675553769
Correlation between Volatile matter and Fixed carbon: -0.001462396461517261
Correlation between Volatile matter and Carbon: 0.01585810873317755
Correlation between Volatile matter and Hydrogen: 0.0018277480245005278
Correlation between Volatile matter and Net calorific value (LHV): -0.1577507345643554
Correlation between Fixed carbon and Carbon: 0.014205582625720905
Correlation between Fixed carbon and Hydrogen: -0.0007166576828273393
Correlation between Fixed carbon and Net calorific value (LHV): -0.11096182610809648
Correlation between Carbon and Hydrogen: -0.008061316044390929
Correlation between Car